In [3]:
from typing import List

## 75. Sort Colors

In [3]:
# first count, then write (2 passes)
# O(n) for time; O(1) since it's in-place.
class Solution:
    def sortColors(self, nums: List[int]) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        if not nums:
            return
        n = len(nums)
        red = 0
        white = 0
        blue = 0
        for i in range(n):
            if nums[i] == 0:
                red += 1
            elif nums[i]  == 1:
                white += 1
            else:
                blue += 1
        i = 0
        while red > 0:
            nums[i] = 0
            red -= 1
            i += 1
        while white > 0:
            nums[i] = 1
            white -= 1
            i += 1
        while blue > 0:
            nums[i] = 2
            blue -= 1
            i += 1

In [4]:
# How to do it in one pass
# use pointers

# tricky part: my previous solution failed because I update the curr pointer in a for loop, so it +1 every iteration.
# In fact should use a while loop here and choose whether to update the curr pointer according to conditions.
# The difference is not to move curr pointer to the right after switching with one of the right numbers.
# So the case [1,2,0] would not be [1,0,2] eventually.

class Solution:
    def sortColors(self, nums: List[int]) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        if not nums:
            return
        n = len(nums)
        red = 0
        blue = n - 1
        curr = 0
        while curr <= blue:
            if nums[curr] == 0:
                nums[red], nums[curr] = nums[curr], nums[red]
                red += 1
                curr += 1
            elif nums[curr] == 1:
                curr += 1
            else:
                nums[blue], nums[curr] = nums[curr], nums[blue]
                blue -= 1
                # do not update curr here because new nums[curr] could be 0, and has
                # to be switched to the left again

### Write a sorting function

## 347. Top K Frequent Elements

In [9]:
nums = [1,2,3]
nums.sort(reverse=True)
print(nums)

[3, 2, 1]


In [10]:
# my solution:
# time: O(n + k + klogk + n) -> O(nlogn)
# space: O(n)
class Solution:
    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        output = []
        # build a hash map of nums -> count
        hashmap = dict()
        for num in nums:
            if num in hashmap:
                hashmap[num] += 1
            else:
                hashmap[num] = 1
        
        # get a hash map of count -> nums (a list of nums)
        # note that some nums may have same counts (case [1,2], 2)
        reverse = dict()
        for key, value in hashmap.items():
            if value not in reverse:
                reverse[value] = [key]
            else:
                reverse[value].append(key)
        cnts = list(reverse.keys())  # list of distinct cnts
        cnts.sort(reverse=True)
        i = 0
        while len(output) < k:
            output += reverse[cnts[i]]
            i += 1
        return output

### How to use heap in python:
https://hg.python.org/cpython/file/2.7/Lib/heapq.py#l16  
Time to add an element to a heap of size k is O(logk). Do it n times will result in O(nlogk).

### Counter in Python:

In [17]:
# Heap solution:
# better than O(nlogn): O(nlogk) or O(n) of time
from collections import Counter
import heapq
nums = [1,1,1,2,3,3]
k = 2

if k == len(nums):
    print(nums)
count = Counter(nums)
print(count)
print(count.get)
print(heapq.nlargest(k, count.keys(), key=count.get))

Counter({1: 3, 3: 2, 2: 1})
<built-in method get of Counter object at 0x7fb98f79bc50>
[1, 3]


In [18]:
# bucket sort (in the comment)
# only O(n) time
nums = [1,1,1,2,3,3]
k = 2
bucket = [[] for _ in range(len(nums) + 1)]
Count = Counter(nums).items()  
for num, freq in Count: bucket[freq].append(num)
print(bucket)
flat_list = [item for sublist in bucket for item in sublist]
print(flat_list[::-1][:k])

[[], [2], [3], [1], [], [], []]
[1, 3]


In [20]:
# list comprehension
bucket = [[], [2], [3], [1]]
flat = [item for sublist in bucket for item in sublist]
print(flat)

[2, 3, 1]


## 215. Kth Largest Element in an Array

In [21]:
# sort
# time O(nlogn) and space O(1)
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        if not nums:
            return -1
        nums.sort(reverse=True)
        return nums[k-1]

In [23]:
# heap
# time O(nlogk) and space O(k)
nums = [3,2,1,5,6,4]
k = 2
curr = heapq.nlargest(k, nums)  # the function returns in descending order
print(curr)
print(curr[-1])

[6, 5]
5


In [1]:
# quickselect:
# would have average time of O(n), worst case O(n^2)
# leetcode solution:
class Solution:
    def findKthLargest(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        def partition(left, right, pivot_index):
            pivot = nums[pivot_index]
            # 1. move pivot to end
            nums[pivot_index], nums[right] = nums[right], nums[pivot_index]  
            
            # 2. move all smaller elements to the left
            store_index = left
            for i in range(left, right):
                if nums[i] < pivot:
                    nums[store_index], nums[i] = nums[i], nums[store_index]
                    store_index += 1

            # 3. move pivot to its final place
            nums[right], nums[store_index] = nums[store_index], nums[right]  
            
            return store_index
        
        def select(left, right, k_smallest):
            """
            Returns the k-th smallest element of list within left..right
            """
            if left == right:       # If the list contains only one element,
                return nums[left]   # return that element
            
            # select a random pivot_index between 
            pivot_index = random.randint(left, right)     
                            
            # find the pivot position in a sorted list   
            pivot_index = partition(left, right, pivot_index)
            
            # the pivot is in its final sorted position
            if k_smallest == pivot_index:
                 return nums[k_smallest]
            # go left
            elif k_smallest < pivot_index:
                return select(left, pivot_index - 1, k_smallest)
            # go right
            else:
                return select(pivot_index + 1, right, k_smallest)

        # kth largest is (n - k)th smallest 
        return select(0, len(nums) - 1, len(nums) - k)

## 162. Find Peak Element

In [4]:
# binary search: O(logn) in time
# For space complexity, because it's recursive solution, the depth of recursion tree will take O(logn).
# If written as iterative version, space complexity will be O(1) instead.
class Solution:
    def search(self, nums, left, right):
        if left == right:
            return left
        mid = (left + right) // 2  # in python3, '/' would return true division
        if nums[mid] < nums[mid+1]:
            return self.search(nums, mid+1, right)
        else:
            return self.search(nums, left, mid)
    def findPeakElement(self, nums: List[int]) -> int:
        if not nums or len(nums) == 0:
            return None
        return self.search(nums, 0, len(nums) - 1)

## 34. Find First and Last Position of Element in Sorted Array (Search for a Range)

In [5]:
# my answer:
# first use binary search to find a single instance of target, then expand linearly to explore its boundaries
# time: O(logn + k), k is the duplicate number of target in nums
# space: O(1)
class Solution:
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        if len(nums) == 0:
            return [-1,-1]
        # binary search for one target
        probe = -1
        left = 0
        right = len(nums) - 1
        while left < right:
            mid = (left + right) // 2
            if nums[mid] == target:
                probe = mid  # record it
                break
            elif nums[mid] < target:
                left = mid + 1
            else:
                right = mid  # not mid-1, because mid-1 may < 0 or right < left (since mid is more close to the left)
        if probe == -1:
            # then must left == right
            if nums[left] != target:
                return [-1,-1]
            else:
                probe = left
        # otherwise, target is found and is stored in probe
        # find the boundaries
        left_bound = probe
        right_bound = probe
        while left_bound > 0:
            if nums[left_bound-1] == target:
                left_bound -= 1
            else:
                break
        while right_bound < len(nums) - 1:
            if nums[right_bound+1] == target:
                right_bound += 1
            else:
                break
        return [left_bound, right_bound]